In [93]:
import pandas as pd
import numpy as np
import seaborn as sns


#### Import the data

In [94]:
customer_info = pd.read_excel('data/Bank_Churn_Messy.xlsx', sheet_name='Customer_Info')
account_info = pd.read_excel('data/Bank_Churn_Messy.xlsx', sheet_name='Account_Info')

In [95]:
customer_info.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,EstimatedSalary
0,15634602,Hargrave,619,FRA,Female,42.0,2,€101348.88
1,15647311,Hill,608,Spain,Female,41.0,1,€112542.58
2,15619304,Onio,502,French,Female,42.0,8,€113931.57
3,15701354,Boni,699,FRA,Female,39.0,1,€93826.63
4,15737888,Mitchell,850,Spain,Female,43.0,2,€79084.1


In [96]:
account_info.head()

,CustomerId,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,€0.0,1,Yes,2,Yes,1
1,15634602,€0.0,1,Yes,2,Yes,1
2,15647311,€83807.86,1,Yes,1,Yes,0
3,15619304,€159660.8,3,No,8,No,1
4,15701354,€0.0,2,No,1,No,0


#### Join "Account_Info" to "Customer_Info"

In [97]:
customers = customer_info.merge(
    account_info,
    how="left",
    on="CustomerId"
)

#### Removing duplicates

In [98]:
customers.duplicated().any()

np.True_

In [99]:
customers[customers.duplicated()]

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure_x,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure_y,IsActiveMember,Exited
1,15634602,Hargrave,619,FRA,Female,42.0,2,€101348.88,€0.0,1,Yes,2,Yes,1
10001,15628319,Walker,792,French,Female,28.0,4,€38190.78,€130142.79,1,No,4,No,0
10002,15628319,Walker,792,French,Female,28.0,4,€38190.78,€130142.79,1,No,4,No,0
10003,15628319,Walker,792,French,Female,28.0,4,€38190.78,€130142.79,1,No,4,No,0


In [100]:
customers = customers.drop_duplicates().reset_index(drop=True)

In [114]:
customers.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure_x,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure_y,IsActiveMember,Exited
0,15634602,Hargrave,619,FRA,Female,42,2,101348.88,0.00,1,True,2,True,True
1,15647311,Hill,608,Spain,Female,41,1,112542.58,83807.86,1,True,1,True,False
2,15619304,Onio,502,French,Female,42,8,113931.57,159660.80,3,False,8,False,True
3,15701354,Boni,699,FRA,Female,39,1,93826.63,0.00,2,False,1,False,False
4,15737888,Mitchell,850,Spain,Female,43,2,79084.10,125510.82,1,True,2,True,False


#### Check the data types for each column and make any necessary fixes

In [113]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  object 
 1   Surname          9997 non-null   object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              9997 non-null   Int64  
 6   Tenure_x         10000 non-null  int64  
 7   EstimatedSalary  9997 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  bool   
 11  Tenure_y         10000 non-null  int64  
 12  IsActiveMember   10000 non-null  bool   
 13  Exited           10000 non-null  bool   
dtypes: Int64(1), bool(3), float64(2), int64(4), object(4)
memory usage: 898.6+ KB


In [103]:
customers.Age = customers.Age.round().astype('Int64')

customers.EstimatedSalary = customers.EstimatedSalary.str.strip().str.strip('€')
customers.EstimatedSalary = pd.to_numeric(customers.EstimatedSalary,errors="coerce")

customers.Balance = customers.Balance.str.strip().str.strip('€')
customers.Balance = pd.to_numeric(customers.Balance,errors="coerce")

In [104]:
customers.HasCrCard = customers.HasCrCard.str.lower().str.replace('yes','1').str.replace('no','0')
customers.HasCrCard = pd.to_numeric(customers.HasCrCard, errors="coerce")
customers.HasCrCard = customers.HasCrCard.astype(bool)

In [107]:
customers.IsActiveMember = customers.IsActiveMember.str.lower().str.replace('yes','1').str.replace('no','0')
customers.IsActiveMember = pd.to_numeric(customers.IsActiveMember, errors="coerce")
customers.IsActiveMember = customers.IsActiveMember.astype(bool)

In [110]:
customers.Exited = customers.Exited.astype(bool)

In [112]:
customers.CustomerId = customers.CustomerId.astype(str)

#### Replace missing values in categorical columns with "MISSING", and missing values in numeric columns with the median

In [120]:
customers.isna().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure_x           0
EstimatedSalary    0
Balance            0
NumOfProducts      0
HasCrCard          0
Tenure_y           0
IsActiveMember     0
Exited             0
dtype: int64

In [116]:
customers.Surname = np.where(customers.Surname.isna(),'MISSING',customers.Surname)

In [117]:
customers.Age = np.where(customers.Age.isna(),customers.Age.median(),customers.Age)

In [119]:
customers.EstimatedSalary = np.where(customers.EstimatedSalary.isna(),customers.EstimatedSalary.median(),customers.EstimatedSalary)

#### Profile the numeric columns in the data. Are there any extreme or non-sensical values? If so, impute them with the median of the column.